## NeuroDet: split data by classes

### Import Libraries

In [17]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import PIL
import PIL.Image
import matplotlib.pyplot as plt
import pathlib
import math

# Disabling GPU for the moment because of the lack of the memory
os.environ["CUDA_VISIBLE_DEVICES"] = "-1" 
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

ModuleNotFoundError: No module named 'tensorflow.keras.layers.TimeDistributed'

### Loading the dataset of Brain Scan images
#### Full data for training and testing
Source of the Dataset: [Kaggle-Brain Tumor Classification](https://www.kaggle.com/sartajbhuvaji/brain-tumor-classification-mri?select=Testing)<br>
Reference for operations performed : [Tensorflow tutorial: Load Images](https://www.tensorflow.org/tutorials/load_data/images)

In [3]:
categories_path = {'glioma_tumor': '/glioma_tumor', 'meningioma_tumor': '/meningioma_tumor', 
                   'pituitary_tumor': '/pituitary_tumor', 'no_tumor':'/no_tumor'}
train_path = 'BrainMRI/Training'
test_path = 'BrainMRI/Testing'

# train_glioma_dir = pathlib.Path(train_path + categories_path['glioma_tumor'])
train_dir = pathlib.Path(train_path)
test_dir = pathlib.Path(test_path)

# Training data: number of examples
# label = 0 | glioma_tumor: 826
# label = 1 | meningioma_tumor: 822
# label = 2 | no_tumor: 395 
# label = 3 | pituitary_tumor: 827 

# Testing data: number of examples
# label = 0 | glioma_tumor: 100
# label = 1 | meningioma_tumor: 115
# label = 2 | no_tumor: 105
# label = 3 | pituitary_tumor: 74

num_train_examples_dict = {
    "label_0": 826, "label_1": 822, "label_2": 395, "label_3": 827}

num_test_examples_dict = {
    "label_0": 100, "label_1": 115, "label_2": 105, "label_3": 74}


num_train_examples = sum(num_train_examples_dict.values()) # 2870
num_test_examples = sum(num_test_examples_dict.values()) # 394

print(f'Total number of train examples: {num_train_examples}')
print(f'Total number of test examples: {num_test_examples}') 

# Defining the parameters of the dataset
batch_size = 128
img_height = 256 # was 128
img_width = 256 # was 128



# Loading the train dataset using keras.utils.image_dataset_from_directory
# To use this method, please ensure you have tf.nigthly installed 
train_data_full = tf.keras.utils.image_dataset_from_directory(
    train_dir, seed=123,
    image_size=(img_height, img_width),
    batch_size=num_train_examples, 
    shuffle=True,
    color_mode = 'grayscale')

test_data_full = tf.keras.utils.image_dataset_from_directory(
    test_dir, seed=123,
    image_size=(img_height, img_width),
    batch_size=num_test_examples, 
    shuffle=True,
    color_mode = 'grayscale')

print(f'train_data_full len = {len(train_data_full)}') # 1 batch = full dataset
print(f'test_data_full len = {len(test_data_full)}') # 1 batch = full dataset
print(train_data_full.class_names)
print(test_data_full.class_names)


Total number of train examples: 2870
Total number of test examples: 394
Found 2870 files belonging to 4 classes.
Found 394 files belonging to 4 classes.
train_data_full len = 1
test_data_full len = 1
['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']


2021-12-07 21:17:28.331009: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Normalizing the training and testing set

In [4]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_data_full = train_data_full.map(lambda x, y: (normalization_layer(x), y))
test_data_full = test_data_full.map(lambda x, y: (normalization_layer(x), y))

# FYI: will print error b/c you can't print class_names after normalizing with .map
# print(train_ds.class_names) 
# print(test_ds.class_names) 

### Function to split data by classes

In [127]:
def split_data_classes(dataset):
    images_0 = []; labels_0 = []
    images_1 = []; labels_1 = []
    images_2 = []; labels_2 = []
    images_3 = []; labels_3 = []
    # OR can do:
    # for batch_images, batch_labels in train_ds_unbatched: 
    #    print(f'batch_images = {batch_images.shape}') # (2870, 256, 256, 1)
    #    print(f'batch_labels = {batch_labels.shape}') # (2870,)
    # for i in range(len(batch_labels): ...

    for batch_data in dataset: 
        batch_images = batch_data[0]
        batch_labels = batch_data[1]
    
        for i in range(len(batch_labels)):
            if batch_labels[i] == 0:
                images_0.append(batch_images[i])
                labels_0.append(batch_labels[i])
            elif batch_labels[i] == 1:
                images_1.append(batch_images[i])
                labels_1.append(batch_labels[i])
            elif batch_labels[i] == 2:
                images_2.append(batch_images[i])
                labels_2.append(batch_labels[i]) 
            else: # batch_labels[i] == 3:
                images_3.append(batch_images[i])
                labels_3.append(batch_labels[i])
    return [images_0, images_1, images_2, images_3, labels_0, labels_1, labels_2, labels_3]

# train_images_0, train_images_1, train_images_2, train_images_3, train_labels_0, train_labels_1, train_labels_2, labels_3 = 
train_data_classes = split_data_classes(dataset=train_ds_unbatched) 
train_images_0 = train_data_classes[0]; train_labels_0 = train_data_classes[4]
train_images_1 = train_data_classes[1]; train_labels_1 = train_data_classes[5]
train_images_2 = train_data_classes[2]; train_labels_2 = train_data_classes[6]
train_images_3 = train_data_classes[3]; train_labels_3 = train_data_classes[7]

# print(train_images_0) # tf of 826 images of shape=(256, 256, 1)
# print(train_labels_1) # tf.Tensor of dtype=int32, all values of 0
print(f'len(train_images_0) = {len(train_images_0)}, len(train_labels_0) = {len(train_labels_0)}') # 826
print(f'len(train_images_1) = {len(train_images_1)}, len(train_labels_1) = {len(train_labels_1)}') # 822
print(f'len(train_images_2) = {len(train_images_2)}, len(train_labels_2) = {len(train_labels_2)}') # 395
print(f'len(train_images_3) = {len(train_images_3)}, len(train_labels_3) = {len(train_labels_3)}') # 827
# 826 + 822 + 395 + 827 = 2870

test_data_classes = split_data_classes(dataset=test_ds_unbatched)
test_images_0 = test_data_classes[0]; test_labels_0 = test_data_classes[4]
test_images_1 = test_data_classes[1]; test_labels_1 = test_data_classes[5]
test_images_2 = test_data_classes[2]; test_labels_2 = test_data_classes[6]
test_images_3 = test_data_classes[3]; test_labels_3 = test_data_classes[7]

print(f'len(test_images_0) = {len(test_images_0)}, len(test_labels_0) = {len(test_labels_0)}') # 100
print(f'len(test_images_1) = {len(test_images_1)}, len(test_labels_1) = {len(test_labels_1)}') # 115
print(f'len(test_images_2) = {len(test_images_2)}, len(test_labels_2) = {len(test_labels_2)}') # 105
print(f'len(test_images_3) = {len(test_images_3)}, len(test_labels_3) = {len(test_labels_3)}') # 74

len(train_images_0) = 826, len(train_labels_0) = 826
len(train_images_1) = 822, len(train_labels_1) = 822
len(train_images_2) = 395, len(train_labels_2) = 395
len(train_images_3) = 827, len(train_labels_3) = 827
len(test_images_0) = 100, len(test_labels_0) = 100
len(test_images_1) = 115, len(test_labels_1) = 115
len(test_images_2) = 105, len(test_labels_2) = 105
len(test_images_3) = 74, len(test_labels_3) = 74


### LSTM for full data

In [5]:
for images, labels in train_data_full:
    x_train = images
    y_train = labels
    
for images, labels in test_data_full:
    x_test = images
    y_test = labels

print(f'x_train.shape = {x_train.shape}') # (2870, 256, 256, 1)
print(f'y_train.shape = {y_train.shape}') # (2870,)
print(f'x_test.shape = {x_test.shape}') # (394, 256, 256, 1)
print(f'y_test.shape = {y_test.shape}') # (394,)

print(f'x_train.shape[0] = {x_train.shape[0]}')

x_train.shape = (2870, 256, 256, 1)
y_train.shape = (2870,)
x_test.shape = (394, 256, 256, 1)
y_test.shape = (394,)
x_train.shape[0] = 2870


In [167]:
batch_size = 32

# Subset data so that it is a multiple of batch_size
def subset_data(data, batch_size, dataset_type):
    data_subset_num_examples = (data.shape[0]//batch_size)*batch_size
    if dataset_type == 'images':
        data_subset = data[:data_subset_num_examples,:,:,:] 
    else: # dataset_type == 'labels'
        data_subset = data[:data_subset_num_examples,]       
    return data_subset

x_train = subset_data(x_train, train_batch_size,'images')
print(f'x_train.shape = {x_train.shape}')

y_train = subset_data(y_train, train_batch_size, 'labels')
print(f'y_train.shape = {y_train.shape}')

x_test = subset_data(x_test, train_batch_size, 'images')
print(f'y_test.shape = {x_test.shape}')
y_test = subset_data(y_test, train_batch_size, 'labels')
print(f'y_test.shape = {y_test.shape}')


x_train.shape = (2848, 256, 256, 1)
y_train.shape = (2848,)
y_test.shape = (384, 256, 256, 1)
y_test.shape = (384,)


In [168]:
# Subset x_train so that it is a multiple of batch_size
x_train_subset_n = (x_train.shape[0]//batch_size)*batch_size
x_train_subset = x_train[:x_train_subset_n,:,:,:]
print(x_train_subset.shape)
x_train = x_train_subset
print(x_train.shape)


# Subset y_train so that it is a multiple of batch_size
y_train_subset_n = (y_train.shape[0]//batch_size)*batch_size
y_train_subset = y_train[:y_train_subset_n,]
print(y_train_subset.shape)

y_train = y_train_subset
print(y_train.shape)

(2848, 256, 256, 1)
(2848, 256, 256, 1)
(2848,)
(2848,)


## 2 LSTM layers, 1 Dense layer

In [163]:

inputs = tf.keras.layers.Input(batch_input_shape=(batch_size,256,256))

lstm_size = 128
num_epochs = 10 #50 # 5
learning_rate = 0.005
dense1_size = 10

lstm1_output = tf.keras.layers.LSTM(lstm_size, return_sequences=True)(inputs)
lstm2_output = tf.keras.layers.LSTM(lstm_size)(lstm1_output)

outputs = tf.keras.layers.Dense(dense1_size, activation='softmax')(lstm2_output)

model = tf.keras.models.Model(inputs, outputs)



model.summary()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

model.fit(
    tf.squeeze(x_train), 
    tf.squeeze(y_train), 
    epochs=num_epochs, 
    # shuffle=True,
    batch_size=batch_size)

test_loss, test_accuracy = model.evaluate(x_test, y_test, batch_size=batch_size)

print(f'Testing loss = {test_loss}, Testing accuracy = {test_accuracy}')

Model: "model_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(32, 256, 256)]          0         
                                                                 
 lstm_22 (LSTM)              (32, 256, 128)            197120    
                                                                 
 lstm_23 (LSTM)              (32, 128)                 131584    
                                                                 
 dense_22 (Dense)            (32, 10)                  1290      
                                                                 
Total params: 329,994
Trainable params: 329,994
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
88/88 [==============================] - 20s 194ms/step - loss: 1.4126 - accuracy: 0.2805
Epoch 2/10
88/88 [==============================] - 18s 201ms/step - loss: 1.3644 -

## 1 LSTM layer, 1 Dense layer

In [177]:
#inputs = tf.keras.layers.Input(batch_input_shape=(None,256,256))
inputs = tf.keras.layers.Input(batch_input_shape=(batch_size,256,256))

# num_epochs = 10
# s 10:35 - <11:10
# x = tf.keras.layers.LSTM(512)(inputs)
# Epoch 10/10
# 90/90 [==============================] - 179s 2s/step - loss: 0.4806 - acc: 0.7993
# 13/13 [==============================] - 19s 1s/step - loss: 1.9995 - acc: 0.4137
# Testing loss = 1.9995168447494507, Testing accuracy = 0.41370558738708496

# x = tf.keras.layers.LSTM(128)(inputs)
# 0.5636 - acc: 0.7666, 10 epochs, 
# Testing loss = 1.8103610277175903, Testing accuracy = 0.3832487165927887

# 3:04 start
lstm_size = 128
num_epochs = 100 #50 # 5
learning_rate = 0.008
dense1_size = 10

lstm_output = tf.keras.layers.LSTM(lstm_size)(inputs)

# Epoch 50/50
# 90/90 [==============================] - 10s 114ms/step - loss: 0.0911 - acc: 0.9683
# 13/13 [==============================] - 1s 38ms/step - loss: 2.2423 - acc: 0.6827
# Testing loss = 2.2423360347747803, Testing accuracy = 0.682741105556488


#outputs = tf.keras.layers.Dense(train_batch_size, activation='softmax')(x)

outputs = tf.keras.layers.Dense(dense1_size, activation='softmax')(lstm_output)

model = tf.keras.models.Model(inputs, outputs)
model.summary()

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

model.fit(
    tf.squeeze(x_train), 
    tf.squeeze(y_train), 
    epochs=num_epochs, 
    # shuffle=True,
    batch_size=batch_size)

test_loss, test_accuracy = model.evaluate(x_test, y_test, batch_size=batch_size)

print(f'Testing loss = {test_loss}, Testing accuracy = {test_accuracy}')

Model: "model_33"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_34 (InputLayer)       [(32, 256, 256)]          0         
                                                                 
 lstm_34 (LSTM)              (32, 128)                 197120    
                                                                 
 dense_33 (Dense)            (32, 10)                  1290      
                                                                 
Total params: 198,410
Trainable params: 198,410
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
89/89 [==============================] - 10s 95ms/step - loss: 1.4022 - accuracy: 0.2872
Epoch 2/100
89/89 [==============================] - 9s 98ms/step - loss: 1.3614 - accuracy: 0.2823
Epoch 3/100
89/89 [==============================] - 9s 104ms/step - loss: 1.3572 - accuracy: 0.2928
Epoch 4/100
89

## Default

In [182]:
for images, labels in train_data_full:
    x_train = images
    y_train = labels
    
for images, labels in test_data_full:
    x_test = images
    y_test = labels


lstm_size = 128
num_epochs = 50 # 5
learning_rate = 0.001
dense1_size = 10

model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(batch_input_shape=(None,256,256)))
model.add(tf.keras.layers.LSTM(lstm_size))
model.add(tf.keras.layers.Dense(dense1_size, activation='softmax'))

model.summary()

model.compile(
    # optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

model.fit(
    tf.squeeze(x_train), 
    tf.squeeze(y_train), 
    epochs=num_epochs)
    # shuffle=True,
    #batch_size=batch_size)

# test_loss, test_accuracy = model.evaluate(x_test, y_test, batch_size=batch_size)
test_loss, test_accuracy = model.evaluate(x_test, y_test)

print(f'Testing loss = {test_loss}, Testing accuracy = {test_accuracy}')

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_38 (LSTM)              (None, 128)               197120    
                                                                 
 dense_37 (Dense)            (None, 10)                1290      
                                                                 
Total params: 198,410
Trainable params: 198,410
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
90/90 [==============================] - 12s 111ms/step - loss: 1.4797 - accuracy: 0.2669
Epoch 2/50
90/90 [==============================] - 10s 111ms/step - loss: 1.3611 - accuracy: 0.2864
Epoch 3/50
67/90 [=====================>........] - ETA: 2s - loss: 1.3575 - accuracy: 0.2766

KeyboardInterrupt: 

## LSTM works best - idk why

In [186]:
train_ds_unbatched = tf.keras.utils.image_dataset_from_directory(train_dir, seed=123,
  image_size=(img_height, img_width),
  batch_size=2870, color_mode = 'grayscale', shuffle=True)

test_ds_unbatched = tf.keras.utils.image_dataset_from_directory(test_dir, seed=123,
  image_size=(img_height, img_width),
  batch_size=394, color_mode = 'grayscale', shuffle=True)

print(f'train_ds_unbatched len = {len(train_ds_unbatched)}') # 1 batch = full dataset
print(f'test_ds_unbatched len = {len(test_ds_unbatched)}') # 1 batch = full dataset
print(train_ds_unbatched.class_names)
print(test_ds_unbatched.class_names)

normalization_layer = tf.keras.layers.Rescaling(1./255)
print(train_ds.class_names)
print(test_ds.class_names)

train_ds_unbatched = train_ds_unbatched.map(lambda x, y: (normalization_layer(x), y))
test_ds_unbatched = test_ds_unbatched.map(lambda x, y: (normalization_layer(x), y))

x_train = None; y_train = None
for images, labels in train_ds_unbatched:
    x_train = images
    y_train = labels
    
x_test = None; y_test = None
for images, labels in test_ds_unbatched:
    x_test = images
    y_test = labels

inputs = tf.keras.layers.Input(batch_input_shape=(None,256,256))


size_lstm = 128
num_epochs = 50
learning_rate = 0.005
x = tf.keras.layers.LSTM(size_lstm)(inputs)


outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.models.Model(inputs, outputs)
model.summary()
model.compile(#optimizer='adam',
              optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['acc'])

model.fit(tf.squeeze(x_train), tf.squeeze(y_train), epochs=num_epochs)
test_loss, test_acc = model.evaluate(x_test, y_test)

print(f'Testing loss = {test_loss}, Testing accuracy = {test_acc}')

Found 2870 files belonging to 4 classes.
Found 394 files belonging to 4 classes.
train_ds_unbatched len = 1
test_ds_unbatched len = 1
['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
Model: "model_37"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_45 (InputLayer)       [(None, 256, 256)]        0         
                                                                 
 lstm_42 (LSTM)              (None, 128)               197120    
                                                                 
 dense_41 (Dense)            (None, 10)                1290      
                                                                 
Total params: 198,410
Trainable params: 198,410


In [187]:
train_ds_unbatched = tf.keras.utils.image_dataset_from_directory(train_dir, seed=123,
  image_size=(img_height, img_width),
  batch_size=2870, color_mode = 'grayscale', shuffle=True)

test_ds_unbatched = tf.keras.utils.image_dataset_from_directory(test_dir, seed=123,
  image_size=(img_height, img_width),
  batch_size=394, color_mode = 'grayscale', shuffle=True)

print(f'train_ds_unbatched len = {len(train_ds_unbatched)}') # 1 batch = full dataset
print(f'test_ds_unbatched len = {len(test_ds_unbatched)}') # 1 batch = full dataset
print(train_ds_unbatched.class_names)
print(test_ds_unbatched.class_names)

normalization_layer = tf.keras.layers.Rescaling(1./255)
print(train_ds.class_names)
print(test_ds.class_names)

train_ds_unbatched = train_ds_unbatched.map(lambda x, y: (normalization_layer(x), y))
test_ds_unbatched = test_ds_unbatched.map(lambda x, y: (normalization_layer(x), y))

x_train = None; y_train = None
for images, labels in train_ds_unbatched:
    x_train = images
    y_train = labels
    
x_test = None; y_test = None
for images, labels in test_ds_unbatched:
    x_test = images
    y_test = labels

inputs = tf.keras.layers.Input(batch_input_shape=(None,256,256))


size_lstm = 128
num_epochs = 100
learning_rate = 0.005
x = tf.keras.layers.LSTM(size_lstm)(inputs)


outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

model = tf.keras.models.Model(inputs, outputs)
model.summary()
model.compile(#optimizer='adam',
              optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['acc'])

model.fit(tf.squeeze(x_train), tf.squeeze(y_train), epochs=num_epochs)
test_loss, test_acc = model.evaluate(x_test, y_test)

print(f'Testing loss = {test_loss}, Testing accuracy = {test_acc}')

Found 2870 files belonging to 4 classes.
Found 394 files belonging to 4 classes.
train_ds_unbatched len = 1
test_ds_unbatched len = 1
['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
Model: "model_38"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_46 (InputLayer)       [(None, 256, 256)]        0         
                                                                 
 lstm_43 (LSTM)              (None, 128)               197120    
                                                                 
 dense_42 (Dense)            (None, 10)                1290      
                                                                 
Total params: 198,410
Trainable params: 198,410


In [5]:
train_ds_unbatched = tf.keras.utils.image_dataset_from_directory(train_dir, seed=123,
  image_size=(img_height, img_width),
  batch_size=2870, color_mode = 'grayscale', shuffle=True)

test_ds_unbatched = tf.keras.utils.image_dataset_from_directory(test_dir, seed=123,
  image_size=(img_height, img_width),
  batch_size=394, color_mode = 'grayscale', shuffle=True)

print(f'train_ds_unbatched len = {len(train_ds_unbatched)}') # 1 batch = full dataset
print(f'test_ds_unbatched len = {len(test_ds_unbatched)}') # 1 batch = full dataset
print(train_ds_unbatched.class_names)
print(test_ds_unbatched.class_names)

normalization_layer = tf.keras.layers.Rescaling(1./255)

train_ds_unbatched = train_ds_unbatched.map(lambda x, y: (normalization_layer(x), y))
test_ds_unbatched = test_ds_unbatched.map(lambda x, y: (normalization_layer(x), y))

x_train = None; y_train = None
for images, labels in train_ds_unbatched:
    x_train = images
    y_train = labels
    
x_test = None; y_test = None
for images, labels in test_ds_unbatched:
    x_test = images
    y_test = labels


size_lstm = 128
num_epochs = 30
learning_rate = 0.0015

#inputs = tf.keras.layers.Input(batch_input_shape=(None,256,256))
#x = tf.keras.layers.LSTM(size_lstm)(inputs)
#outputs = tf.keras.layers.Dense(10, activation='softmax')(x)
#model = tf.keras.models.Model(inputs, outputs)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(batch_input_shape=(None,256,256)))
model.add(tf.keras.layers.LSTM(size_lstm, return_sequences=True))

model.add(tf.keras.layers.LSTM(size_lstm))
model.add(tf.keras.layers.Dropout(0.01))

model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.01))
model.add(tf.keras.layers.Dense(4, activation='softmax'))

model.summary()
model.compile(#optimizer='adam',
              optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['acc'])

model.fit(tf.squeeze(x_train), tf.squeeze(y_train), epochs=num_epochs)
test_loss, test_acc = model.evaluate(x_test, y_test)

print(f'Testing loss = {test_loss}, Testing accuracy = {test_acc}')

Found 2870 files belonging to 4 classes.
Found 394 files belonging to 4 classes.
train_ds_unbatched len = 1
test_ds_unbatched len = 1
['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 256, 128)          197120    
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 32)                4128      
                                                                 
 dropout_1 (Dropout)         (None, 32)           

above: size_lstm = 128
num_epochs = 30
learning_rate = 0.0015

Epoch 30/30
90/90 [==============================] - 20s 226ms/step - loss: 0.1520 - acc: 0.9470
13/13 [==============================] - 2s 73ms/step - loss: 2.7012 - acc: 0.6269
Testing loss = 2.7011656761169434, Testing accuracy = 0.6269035339355469

In [11]:
train_ds_unbatched = tf.keras.utils.image_dataset_from_directory(train_dir, seed=123,
  image_size=(img_height, img_width),
  batch_size=2870, color_mode = 'grayscale', shuffle=True)

test_ds_unbatched = tf.keras.utils.image_dataset_from_directory(test_dir, seed=123,
  image_size=(img_height, img_width),
  batch_size=394, color_mode = 'grayscale', shuffle=True)

normalization_layer = tf.keras.layers.Rescaling(1./255)

train_ds_unbatched = train_ds_unbatched.map(lambda x, y: (normalization_layer(x), y))
test_ds_unbatched = test_ds_unbatched.map(lambda x, y: (normalization_layer(x), y))

x_train = None; y_train = None
for images, labels in train_ds_unbatched:
    x_train = images
    y_train = labels
    
x_test = None; y_test = None
for images, labels in test_ds_unbatched:
    x_test = images
    y_test = labels


size_lstm = 128
num_epochs = 30
learning_rate = 0.002


model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(batch_input_shape=(None,256,256,1)))
model.add(tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding="same"))
model.add(tf.keras.layers.MaxPooling2D(padding="same"))
model.add(tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding="same"))
model.add(tf.keras.layers.MaxPooling2D(padding="same"))
model.add(tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding="same"))
model.add(tf.keras.layers.MaxPooling2D(padding="same"))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(80, activation='relu'))
model.add(tf.keras.layers.Dense(4, activation='softmax'))



#model.add(tf.keras.layers.Input(batch_input_shape=(None,256,256)))
#model.add(tf.keras.layers.LSTM(size_lstm, return_sequences=True))

#model.add(tf.keras.layers.LSTM(size_lstm))
#model.add(tf.keras.layers.Dropout(0.01))

#model.add(tf.keras.layers.Dense(32, activation='relu'))
#model.add(tf.keras.layers.Dropout(0.01))
#model.add(tf.keras.layers.Dense(4, activation='softmax'))

model.summary()
model.compile(#optimizer='adam',
              optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['acc'])

model.fit(x_train, y_train, epochs=num_epochs)
test_loss, test_acc = model.evaluate(x_test, y_test)

print(f'Testing loss = {test_loss}, Testing accuracy = {test_acc}')

Found 2870 files belonging to 4 classes.
Found 394 files belonging to 4 classes.
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 256, 256, 10)      100       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 128, 128, 10)     0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 128, 128, 10)      910       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 64, 64, 10)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 64, 64, 10)        910       
                                       

Epoch 30/30
90/90 [==============================] - 22s 245ms/step - loss: 3.3341e-05 - acc: 1.0000
13/13 [==============================] - 1s 76ms/step - loss: 5.8118 - acc: 0.7690
Testing loss = 5.811763763427734, Testing accuracy = 0.7690355181694031

In [24]:
train_ds_unbatched = tf.keras.utils.image_dataset_from_directory(train_dir, seed=123,
  image_size=(img_height, img_width),
  batch_size=2870, color_mode = 'grayscale', shuffle=True)

test_ds_unbatched = tf.keras.utils.image_dataset_from_directory(test_dir, seed=123,
  image_size=(img_height, img_width),
  batch_size=394, color_mode = 'grayscale', shuffle=True)

normalization_layer = tf.keras.layers.Rescaling(1./255)

train_ds_unbatched = train_ds_unbatched.map(lambda x, y: (normalization_layer(x), y))
test_ds_unbatched = test_ds_unbatched.map(lambda x, y: (normalization_layer(x), y))

x_train = None; y_train = None
for images, labels in train_ds_unbatched:
    x_train = images
    y_train = labels
    
x_test = None; y_test = None
for images, labels in test_ds_unbatched:
    x_test = images
    y_test = labels


size_lstm = 128
num_epochs = 10
learning_rate = 0.002

from keras.layers import TimeDistributed

cnn = tf.keras.models.Sequential()


cnn.add(tf.keras.layers.Input(batch_input_shape=(None,256,256,1)))
cnn.add(tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding="same"))
cnn.add(tf.keras.layers.MaxPooling2D(padding="same"))
cnn.add(tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding="same"))
cnn.add(tf.keras.layers.MaxPooling2D(padding="same"))
cnn.add(tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding="same"))
cnn.add(tf.keras.layers.MaxPooling2D(padding="same"))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(80, activation='relu'))
cnn.add(tf.keras.layers.Dense(4, activation='softmax'))


model = tf.keras.models.Sequential()
model.add(tf.keras.layers.TimeDistributed(cnn))

#model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Input(batch_input_shape=(None,256,256,1))))
#model.add(TimeDistributed(tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding="same")))
#model.add(TimeDistributed(tf.keras.layers.MaxPooling2D(padding="same")))
#model.add(TimeDistributed(tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding="same")))
#model.add(TimeDistributed(tf.keras.layers.MaxPooling2D(padding="same")))
#model.add(TimeDistributed(tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding="same")))
#model.add(TimeDistributed(tf.keras.layers.MaxPooling2D(padding="same")))
#model.add(TimeDistributed(tf.keras.layers.Flatten()))

model.add(tf.keras.layers.LSTM(size_lstm))
model.add(tf.keras.layers.Dense(80, activation='relu'))
model.add(tf.keras.layers.Dense(4, activation='softmax'))



#model.add(tf.keras.layers.Input(batch_input_shape=(None,256,256)))
#model.add(tf.keras.layers.LSTM(size_lstm, return_sequences=True))

#model.add(tf.keras.layers.LSTM(size_lstm))
#model.add(tf.keras.layers.Dropout(0.01))

#model.add(tf.keras.layers.Dense(32, activation='relu'))
#model.add(tf.keras.layers.Dropout(0.01))
#model.add(tf.keras.layers.Dense(4, activation='softmax'))


model.compile(#optimizer='adam',
              optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
             loss='sparse_categorical_crossentropy',
             metrics=['acc'])

model.fit(x_train, y_train, epochs=num_epochs)
model.summary()
test_loss, test_acc = model.evaluate(x_test, y_test)

print(f'Testing loss = {test_loss}, Testing accuracy = {test_acc}')

Found 2870 files belonging to 4 classes.
Found 394 files belonging to 4 classes.
Epoch 1/10


ValueError: in user code:

    File "/Users/hyeyeonhwang/Desktop/CSCI_2470/final_project/neuroDet_env/lib/python3.7/site-packages/keras/engine/training.py", line 946, in train_function  *
        return step_function(self, iterator)
    File "/Users/hyeyeonhwang/Desktop/CSCI_2470/final_project/neuroDet_env/lib/python3.7/site-packages/keras/engine/training.py", line 935, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/hyeyeonhwang/Desktop/CSCI_2470/final_project/neuroDet_env/lib/python3.7/site-packages/keras/engine/training.py", line 928, in run_step  **
        outputs = model.train_step(data)
    File "/Users/hyeyeonhwang/Desktop/CSCI_2470/final_project/neuroDet_env/lib/python3.7/site-packages/keras/engine/training.py", line 841, in train_step
        y_pred = self(x, training=True)
    File "/Users/hyeyeonhwang/Desktop/CSCI_2470/final_project/neuroDet_env/lib/python3.7/site-packages/keras/utils/traceback_utils.py", line 67